In [1]:
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [52]:
import matplotlib.pyplot as plt
import seaborn as sns
from autorocks.viz import plots_setup
import torch
import gpytorch
import botorch.posteriors
from botorch import models
from botorch.optim import fit
import torch
import numpy as np
import pandas as pd
import botorch

output_location = "/Users/salabed/workspace/latex_writings/thesis/phd_dissertation/Chapters/Background/Figures/"
plt.style.use("ggplot")
sns.set_theme(style="ticks", rc={"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex=False)
plt.rc("xtick", labelsize="large")
plt.rc("ytick", labelsize="large")
plt.rc("axes", labelsize="large")
plt.rc("pdf", use14corefonts=True)



class Forrester(botorch.test_functions.SyntheticTestFunction):

    dim = 1
    _bounds = [(0.0, 1.0)]
    _optimal_value = -6.0
    _optimizers = [(0.78)]

    def evaluate_true(self, X: torch.Tensor) -> torch.Tensor:
        return torch.pow(6 * X - 2, 2) * torch.sin(12 * X - 4) 
        
    
problem = Forrester(negate = True)



In [235]:
num_observations = 1
train_x_full, train_y_full = plots_setup.generate_data(num_observations, problem)

acf_map = {
    "EI": botorch.acquisition.ExpectedImprovement,
    "UCB": botorch.acquisition.UpperConfidenceBound,
    "PI": botorch.acquisition.ProbabilityOfImprovement
}


In [360]:
test_x = torch.linspace(0, 1)
real_y = problem(test_x)
predicted = []

train_x_full = torch.tensor([[0.2]])
for acf_name, func in acf_map.items():
    train_x = train_x_full.detach().clone()
    for i in range(1, 13):
        train_y = problem(train_x)
        model = models.FixedNoiseGP(
            train_X=train_x,
            train_Y=train_y,
            train_Yvar=torch.zeros_like(train_y)
        )
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
        fit.fit_gpytorch_scipy(mll)
        cr = plots_setup.predict(model = model, test_x = test_x, observation_noise = False)
        lower, upper, mean = cr.lower, cr.upper, cr.mean
        # Next get the valuation of the acquisition function
        
        # Use the next value and append it to train_x
        if acf_name == "UCB": 
            acf = func(model=model, beta=1)        
        else:
            acf = func(model=model, best_f=train_y.max())
        new_candidate, acf_value = botorch.optim.optimize_acqf(acf, bounds=torch.tensor(problem._bounds).T, q=1, num_restarts = 1, raw_samples=64)

        # Get the whole acf line.
        acf_line = acf(test_x.reshape(-1, 1, 1))
        if i % 3 == 0:
            predicted.append({
                  "x":test_x.detach().squeeze().cpu().numpy() ,
                  # Y: Model prediction
                  "y": mean.detach().squeeze().cpu().numpy(),
                  "low": lower.detach().squeeze().cpu().numpy(),
                  "high": upper.detach().squeeze().cpu().numpy(), 
                  # X_train: The observation so far
                  "x_train": train_x.detach().squeeze().cpu().numpy(),
                  # Y_train: The observation so far and their value
                  "y_train": train_y.detach().squeeze().cpu().numpy(),
                  # x_truth: the full line  value
                  "x_truth": test_x.detach().squeeze().cpu().numpy(),
                  # y_truth: the full line value
                  "y_truth": real_y.detach().squeeze().cpu().numpy(),
                  # ACF_Value: The AF evaluation that led to choosing the cnadidate
                   "acf_value": acf_value.detach().squeeze().cpu().numpy()[np.newaxis, ...],
                  # acf_line: The evaluation of alpha(x), should be plotted with x_truth
                  "acf_line": acf_line.detach().squeeze().cpu().numpy(),
                  # Candidate: the next candidate as suggested by the optimizer
                  "candidate": new_candidate.detach().squeeze().cpu().numpy()[np.newaxis, ...],
                  "Obs": np.array([i]),
                  "AF": np.array([acf_name])})
        train_x = torch.concat([train_x, new_candidate])


In [361]:
df = pd.DataFrame.from_dict(predicted)


In [362]:

# df = df.explode(column=["x", "y", "x_train", "y_train", "x_truth", "y_truth","acf_line"]).reset_index(drop=True)
import itertools

df1 = (df.apply(lambda x: list(itertools.zip_longest(x["x"], x["y"], x["low"], x["high"], x["x_train"], x["y_train"], x["x_truth"], x["y_truth"],x["acf_line"], x["Obs"], x["AF"], x["candidate"], x["acf_value"])), axis=1)
       .explode()
       .apply(lambda x: pd.Series(x, index=["x", "y", "low", "high", "x_train", "y_train", "x_truth", "y_truth", "acf_line", "Obs", "AF", "candidate", "acf_value"]))
       .groupby(level=0).ffill())

df1


In [363]:
DPI = 300  # default dpi for most printers
plt.style.use("ggplot")
sns.set_theme(style="ticks", rc={"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex=False)
plt.rc("xtick", labelsize="small")
plt.rc("ytick", labelsize="small")
plt.rc("axes", labelsize="medium")
plt.rc("pdf", use14corefonts=True)

grid = sns.FacetGrid(df1, row="AF", col="Obs", hue="AF", palette="hls")

# https://seaborn.pydata.org/examples/many_facets.html
grid.map_dataframe(sns.lineplot, "x_truth", "y_truth", label="True Function", alpha=0.4, color = "black", linestyle='--')
grid.map_dataframe(sns.scatterplot, "x_train", "y_train", color='black', marker="*", label="Observed", s=128)
grid.map_dataframe(sns.scatterplot, "candidate", "acf_value", label="Next Candidate", marker="+", color="orange", s=128, linewidth=1.5)
grid.map_dataframe(plt.fill_between, "x", "low", "high", label="CI", alpha=0.1)
grid.map_dataframe(sns.lineplot, "x", "y", label="prediction", alpha=0.7)
grid.set(xlim=(0, 1))
grid.add_legend()

output_format = "svg"
grid.savefig(f"{output_location}/acf_explores.{output_format}", bbox_inches="tight", format=f"{output_format}", dpi=300)


In [359]:
grid = sns.FacetGrid(df1, row="AF", col="Obs", hue='AF', palette="hls", sharey=False)
grid.map_dataframe(sns.scatterplot, "candidate", "acf_value", label="Next Candidate", marker="+", color="orange", s=128, linewidth=1.5)
grid.map_dataframe(sns.lineplot, "x", "acf_line", label="AF(x)")
grid.set_ylabels("AF(x)")
grid.add_legend()

output_format = "svg"
grid.savefig(f"{output_location}/acf_eval.{output_format}", bbox_inches="tight", format=f"{output_format}", dpi=300)
